In [9]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo_search ddgs langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 5.5 MB/s eta 0:00:00a 0:00:01


## Built-in Tool DuckDuckGo Search

In [10]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
result = search.invoke("latest news on aliens")
print(result)

snippet: Get the latest news reports on UFO sightings across the world on the New York Post., title: Ufos | Latest News | New York Post, link: https://nypost.com/tag/ufos/, snippet: Are aliens real and is there life on other planets? Read the latest news and updates here from UFO sightings to theories on life in outer space., title: Aliens - All the latest news, from Area 51 to the latest theories and ..., link: https://metro.co.uk/tag/aliens/, snippet: Stay up to date with the latest UFO and UAP related news , sightings, and developments. Explore breaking stories, expert analysis, and eyewitness accounts., title: Latest - UFO News | The UFO Database, link: https://theufodatabase.com/news, snippet: Follow the search for aliens with the latest extraterrestrial life news , features and articles., title: Extraterrestrial life news, features and articles - Live Science, link: https://www.livescience.com/space/extraterrestrial-life


## Built-in Tool Shell Tool 

In [13]:
from langchain_community.tools import ShellTool
shell_tool = ShellTool()
output = shell_tool.invoke("ls")
print(output)

Executing command:
 ls
sample_data



## Custom Tools

In [34]:
from langchain_core.tools import tool


In [35]:
## Step -1 Create a function 
def Multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [36]:
## Step -2 add Type hints
def Multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [37]:
## Step -3 add tool Decorator

@tool 
def Multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [40]:
result = Multiply.invoke({"a": 3, "b": 4})
print(result)

12


In [42]:
print(Multiply.name)
print(Multiply.description)
print(Multiply.args)

Multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [45]:
print(Multiply.args_schema.model_json_schema())  ## What llm sees .. input tool schema

{'description': 'Multiply two numbers.', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'Multiply', 'type': 'object'}


## Method 2 - Using Structured Tool

In [47]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

In [48]:
class MultiplyInput(BaseModel):
    a : int = Field(required=True, description="The first number to multiply")
    b : int = Field(required=True, description="The second number to multiply")
    

/tmp/ipython-input-3245819144.py:2: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a : int = Field(required=True, description="The first number to multiply")
/tmp/ipython-input-3245819144.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b : int = Field(required=True, description="The second number to multiply")


In [49]:
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [50]:
mulitply_tool = StructuredTool.from_function(
    func=multiply,
    name="Multiply",
    description="Multiplies two numbers together.",
    args_schema=MultiplyInput
)

In [52]:
result = mulitply_tool.invoke({"a": 5, "b": 6})
print(result)

30


## Method 3 Using Base Tool Class

In [53]:
from langchain.tools import BaseTool
from typing import Type

In [54]:
class MultiplyInput(BaseModel):
    a: int = Field(..., description="The first number to multiply")
    b: int = Field(..., description="The second number to multiply")

In [56]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiplies two numbers together."
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [57]:
multiply_tool = MultiplyTool()

In [58]:
result = multiply_tool.invoke({"a": 7, "b": 8})
print(result)

56


## Toolkit

In [59]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [60]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [61]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
